In [20]:
from __future__ import print_function
import os
import cv2
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import numpy as np
import tensorflow as tf
from src.model import Model as Attn_Model
from src.baseline_classify import Model as Clfy_Model
from src.data_loader_pred_classify import dataLoader

In [2]:
width       = 16
height      = 16
max_steps   = 7
batch_size  = 16
print_every = 200
pretrained_clfy_model = './model/clsfy2/model-392'
pretrained_attn_model = './model/lstm1/model-345'

In [3]:
def valid_pred_bboxs(pred_bboxes):
    """
    The first step and last step are start and stop state
    """
    def box_threshold(left, top, width, height):
        valid_box = False
        # If the threshold box is less than 10
        if width * height < 10:
            valid_box = False
        elif int(left) == 0 and int(top) == 0 and\
             int(width) == 0 and int(height) == 0:
            valid_box = False
        else:
            valid_box = True

        return valid_box

    valid_pred_bboxs = []
    for t in range(batch_size):
        interm_pred_bboxs = []
        for T in range(max_steps):
            # Ignore first and last prediction
            if T != 0 and T+1 != max_steps:
                # Predicted bounding box
                smple_left   = abs(pred_bboxes[T][t][0])
                smple_top    = abs(pred_bboxes[T][t][1])
                smple_width  = abs(pred_bboxes[T][t][2])
                smple_height = abs(pred_bboxes[T][t][3])
                # Check if the box is valid
                vld_bbx_pred = box_threshold(left=smple_left, top=smple_top,\
                                     width=smple_width, height=smple_height)
                # Collect if valid box
                if vld_bbx_pred:
                    interm_pred_bboxs.append([smple_left, smple_top, smple_width, smple_height])
        # Collect
        valid_pred_bboxs.append(interm_pred_bboxs)

    return valid_pred_bboxs

def crop_and_resize(images, bboxes):
    images_crop_resize = []
    for t in range(len(bboxes)):
        image = images[t] # Sample image
        bbox  = bboxes[t] # Sample predictions
        interm_images_crop_resize = []
        # Loop through predictions
        for bbx in bbox:
            sample_left   = abs(int(bbx[0]))
            sample_top    = abs(int(bbx[1]))
            sample_width  = abs(int(bbx[2]))
            sample_height = abs(int(bbx[3]))
            # Crop from image
            image_patch = image[sample_top:sample_top+sample_height, sample_left:sample_left+sample_width, :]
            # Zooming
            image_patch_rz = cv2.resize(image_patch, (width, height), interpolation = cv2.INTER_AREA)
            # Normalize image between -1 and 1
            image_patch_rz = image_patch_rz /127.5 - 1.0
            # Collect
            interm_images_crop_resize.append(image_patch_rz)
        # Append to main
        images_crop_resize.append(interm_images_crop_resize)

    return images_crop_resize

In [4]:
# Load train dataset
data = dataLoader(directory='./dataset', dataset_dir='test_cropped',\
                  dataset_name='test.txt', max_steps=7, mode='Test')

All data-loaded!


### -----------------------BBox-Prediction---------------------------------


In [5]:
tf.reset_default_graph()

In [6]:
# Load Attention Model
attn_model = Attn_Model(dim_feature=[49, 128], dim_hidden=128, n_time_step=7,
              alpha_c=1.0, image_height=64, image_width=64, mode='test')

In [7]:
# Set GPU options
config = tf.GPUOptions(allow_growth=True)

# Test dataset
test_loader = data.gen_data_batch(batch_size)
n_examples  = data.max_length
n_iters     = int(np.ceil(float(n_examples)/batch_size))

# Summary
print("Data size:  %d" %n_examples)
print("Batch size: %d" %batch_size)
print("Iterations: %d" %n_iters)

Data size:  13068
Batch size: 16
Iterations: 817


In [8]:
# Build Attention model
pred_bboxs_, _    = attn_model.build_test_model()

CNN build model sucess!


In [9]:
with tf.Session(config=tf.ConfigProto(gpu_options=config)) as sess:
    # Intialize the training graph
    sess.run(tf.global_variables_initializer())
    # Tensorboard summary path
    saver = tf.train.Saver()
    # Load pretrained model
    if pretrained_attn_model is not None:
        print("Start testing with pretrained Attention Model..")
        saver.restore(sess, pretrained_attn_model)
    else:
        print("Start testing with Attention Model with random weights...")

    pred_bboxes = []
    labl_batchs = []
    for i in range(n_iters):
        image_batch, image_norm_batch, label_batch = next(test_loader)
        # Box Prediction
        feed_dict = {attn_model.images: image_norm_batch,
                     attn_model.drop_prob: 1.0}
        prediction_bboxes = sess.run(pred_bboxs_, feed_dict)
        # Collect
        pred_bboxes.append(prediction_bboxes)
        labl_batchs.append(label_batch)
        if i%print_every == 0:
            print('Completion..{%d/%d}' % (i, n_iters))
print('Epoch Completion..{%d/%d}' % (n_iters, n_iters))
print('Completed!')
# Close session
sess.close()

Start testing with pretrained Attention Model..
INFO:tensorflow:Restoring parameters from ./model/lstm1/model-1000
Completion..{0/817}
Completion..{200/817}
Completion..{400/817}
Completion..{600/817}
Completion..{800/817}
Epoch Completion..{817/817}
Completed!


In [10]:
len(pred_bboxes)

817

In [18]:
print(len(pred_bboxes[0]))
print(len(pred_bboxes[0][0]))
print(len(pred_bboxes[0][0]))

7
16
4


In [14]:
print(len(labl_batchs))
print(len(labl_batchs[0]))
print(len(labl_batchs[0][1]))

817
16
3


In [21]:
# Get Valid Predictions boxes
all_images_crop_rez = []
for i in range(len(pred_bboxes)):
    vald_prdct_bbxs = valid_pred_bboxs(pred_bboxes=pred_bboxes[i])
    images_crop_rez = crop_and_resize(images=image_batch, bboxes=vald_prdct_bbxs)
    all_images_crop_rez.append(images_crop_rez)

In [22]:
len(all_images_crop_rez)

817

In [30]:
len(all_images_crop_rez[0])

16

In [47]:
for i in range(len(all_images_crop_rez[0])):
    each_seq = all_images_crop_rez[0][i]
    each_seq = np.array(each_seq)
    print(each_seq.shape)
    #rint(len(each_seq))

(1, 16, 16, 3)
(3, 16, 16, 3)
(1, 16, 16, 3)
(1, 16, 16, 3)
(1, 16, 16, 3)
(2, 16, 16, 3)
(3, 16, 16, 3)
(1, 16, 16, 3)
(3, 16, 16, 3)
(2, 16, 16, 3)
(2, 16, 16, 3)
(2, 16, 16, 3)
(3, 16, 16, 3)
(2, 16, 16, 3)
(2, 16, 16, 3)
(2, 16, 16, 3)


### ------------------------Digit-Classification--------------------------

In [60]:
# Reset graph
tf.reset_default_graph()
# Load Classification Model
clfy_model = Clfy_Model(image_height=16, image_width=16, mode='test')
# Predictions
_, predictions  = clfy_model.build_test_model()

Classification build model sucess!


In [61]:
# To load
saver = tf.train.Saver()

In [63]:
with tf.Session(config=tf.ConfigProto(gpu_options=config)) as sess:
    # Intialize the training graph
    sess.run(tf.global_variables_initializer())
    # Load pretrained model
    if pretrained_clfy_model is not None:
        print("Start testing with Classification pretrained Model..")
        saver.restore(sess, pretrained_clfy_model)
    else:
        print("Start testing with Classification Model with random weights...")
    # Run predictions
    empty=[]
    all_prdcts = []
    for t in range(n_iters):
        each_prd_batch = []
        for T in range(batch_size):
            each_pred = []
            input_seq = np.array(all_images_crop_rez[t][T])
            input_seq = np.array(input_seq)
            try:
                # Test if any sequence exist
                input_seq[0][0]
                # Predictions
                feed_dict = {clfy_model.images: input_seq,
                             clfy_model.drop_prob: 1.0}
                pred = sess.run(predictions, feed_dict)
                each_pred.append(pred)
            except IndexError:
                each_pred.append(empty)
                print('Index Error Occured at: ',t, T)
            # Collect batch outputs
            each_prd_batch.append(each_pred)
        # Collect all outputs
        all_prdcts.append(each_prd_batch)
        # Print Every
        if t%print_every == 0:
            print('Completion..{%d/%d}' % (t, n_iters))

print('Epoch Completion..{%d/%d}' % (n_iters, n_iters))
print('Completed!')
# Close session
sess.close()

Start testing with Classification pretrained Model..
INFO:tensorflow:Restoring parameters from ./model/clsfy2/model-392
Completion..{15/817}
Completion..{15/817}
Completion..{15/817}
Completion..{15/817}
Completion..{15/817}
Epoch Completion..{817/817}
Completed!


In [64]:
len(all_prdcts[0])

16

In [73]:
labl_batchs[0][1]

[2.0, 1.0, 0.0]

In [71]:
all_prdcts[0][1][0]

array([2, 1, 0])

In [77]:
al = all_prdcts[0]
print(len(al))
a2 = al[0][0]
print(a2)

16
[2]


### ----------------------Compute Sequence Accuracy------------------------

In [84]:
total_acc = 0.0
total_seq = 0.0
total_mis = 0.0
for t in range(len(labl_batchs)):
    each_lbl_batch = labl_batchs[t]
    each_prd_batch = all_prdcts[t]
    for T in range(len(each_lbl_batch)):
        each_labl_seq = each_lbl_batch[T]
        try :
            each_pred_seq = each_prd_batch[T][0]
            seq_accuracy  = 0
            for tT in range(len(each_labl_seq)):
                each_labl = each_labl_seq[tT]
                try:
                    each_pred = each_pred_seq[tT]
                    if each_labl == each_pred:
                        seq_accuracy += 1
                except IndexError:
                    print('Index Error Occured at: ', t, T, tT)
                    print(each_labl_seq, ', ', each_pred_seq)
                    print('----------------------------------')
                    total_mis += 1
            # Get the accuracy of sequence
            seq_accuracy = seq_accuracy/len(each_labl_seq)
            # Update sequence
            total_seq += 1
            # Update
            total_acc += seq_accuracy
        except IndexError:
            print('Index Error Occured at: ',t, T)

print('Total misses: ', total_mis)
print('Final Sequence accuracy is: %f' % (total_acc/total_seq))

Index Error Occured at:  0 7 1
[6.0, 5.0] ,  [4]
----------------------------------
Index Error Occured at:  1 4 1
[1.0, 2.0, 9.0] ,  [3]
----------------------------------
Index Error Occured at:  1 4 2
[1.0, 2.0, 9.0] ,  [3]
----------------------------------
Index Error Occured at:  1 14 1
[5.0, 1.0] ,  [6]
----------------------------------
Index Error Occured at:  2 11 3
[1.0, 7.0, 1.0, 2.0] ,  [1 6 7]
----------------------------------
Index Error Occured at:  4 5 1
[2.0, 0.0] ,  [2]
----------------------------------
Index Error Occured at:  4 7 1
[3.0, 2.0] ,  [4]
----------------------------------
Index Error Occured at:  5 13 2
[1.0, 0.0, 2.0] ,  [3 0]
----------------------------------
Index Error Occured at:  7 14 1
[9.0, 6.0] ,  [6]
----------------------------------
Index Error Occured at:  9 11 3
[1.0, 2.0, 9.0, 8.0] ,  [1 5 7]
----------------------------------
Index Error Occured at:  10 3 2
[2.0, 8.0, 9.0] ,  [1 1]
----------------------------------
Index Error Occure

Index Error Occured at:  491 8 1
[2.0, 3.0] ,  [1]
----------------------------------
Index Error Occured at:  491 9 3
[1.0, 9.0, 0.0, 0.0] ,  [1 3 6]
----------------------------------
Index Error Occured at:  491 12 3
[1.0, 3.0, 2.0, 3.0] ,  [1 2 1]
----------------------------------
Index Error Occured at:  493 10 1
[1.0, 2.0] ,  [0]
----------------------------------
Index Error Occured at:  493 11 1
[5.0, 7.0] ,  [2]
----------------------------------
Index Error Occured at:  494 5 1
[1.0, 7.0] ,  [2]
----------------------------------
Index Error Occured at:  494 6 1
[9.0, 4.0] ,  [5]
----------------------------------
Index Error Occured at:  495 2 3
[2.0, 1.0, 7.0, 6.0] ,  [1 5 7]
----------------------------------
Index Error Occured at:  496 4 2
[3.0, 7.0, 3.0] ,  [7 9]
----------------------------------
Index Error Occured at:  497 1 1
[2.0, 1.0, 2.0] ,  [7]
----------------------------------
Index Error Occured at:  497 1 2
[2.0, 1.0, 2.0] ,  [7]
---------------------------